In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import django
import os

os.environ['DJANGO_SETTINGS_MODULE'] = 'dashboard.settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [20]:
import pandas as pd
from data.voterdb import VoterDb

In [21]:
db = VoterDb()

In [90]:
results = db.fetchall(f"""
    select * from voter_history_summary where county_code='060'
""")
len(results)

652815

In [91]:
df = pd.DataFrame.from_records(results, columns=['voter_id', 'county_code',
                                                 'd2014_05_20', 'd2014_11_04',
                                                 'd2016_05_24', 'd2016_11_08',
                                                 'd2018_05_22', 'd2018_11_06',
                                                 'd2020_06_09', 'd2020_11_03',
                                                 'd2022_05_24'])
print(f'total fulton voters = {len(df.index)}')

total fulton voters = 652815


In [92]:
def only_one(row):
    for x in row[2:9]:
        if x is None:
            continue
        if x.find('X') == -1:
            return False
    return True

In [93]:
df1 = df[df.d2020_11_03 == 'GG']
print(f'total fulton voters casting a ballot on 2020-11-3 = {len(df1.index)}')
df2 = df1.assign(only_2020=df1.apply(lambda row: only_one(row), axis=1))
df3 = df2[df2.only_2020]
print(f'total fulton voters with no history prior to 2020-11-3 = {len(df3.index)}')
df4 = df3[df3.d2018_11_06 == 'XG']
print(f'Of the no history voters, the number of voters with opportunity for vote in 2018 general election = {len(df4.index)}')
print(f'turnout % fulton = {len(df1.index)/len(df.index)}')
print(f'% no history fulton = {len(df4.index)/len(df1.index)}')


total fulton voters casting a ballot on 2020-11-3 = 523511
total fulton voters with no history prior to 2020-11-3 = 89819
Of the no history voters, the number of voters with opportunity for vote in 2018 general election = 44829
turnout % fulton = 0.8019285708814902
% no history fulton = 0.0856314384989045


In [94]:
results_cobb = db.fetchall(f"""
    select * from voter_history_summary where county_code='033'
""")

df_cobb = pd.DataFrame.from_records(results_cobb, columns=['voter_id', 'county_code',
                                                 'd2014_05_20', 'd2014_11_04',
                                                 'd2016_05_24', 'd2016_11_08',
                                                 'd2018_05_22', 'd2018_11_06',
                                                 'd2020_06_09', 'd2020_11_03',
                                                 'd2022_05_24'])
print(f'total cobb voters = {len(df_cobb.index)}')

total cobb voters = 474865


In [95]:
df1_cobb = df_cobb[df_cobb.d2020_11_03 == 'GG']
print(f'total cobb voters casting a ballot on 2020-11-3 = {len(df1_cobb.index)}')
df2_cobb = df1_cobb.assign(only_2020=df1_cobb.apply(lambda row: only_one(row), axis=1))
df3_cobb = df2_cobb[df2_cobb.only_2020]
print(f'total cobb voters with no history prior to 2020-11-3 = {len(df3_cobb.index)}')
df4_cobb = df3_cobb[df3_cobb.d2018_11_06 == 'XG']
print(f'Of the no history voters, the number of voters with opportunity for vote in 2018 general election = {len(df4_cobb.index)}')
print(f'turnout % cobb = {len(df1_cobb.index)/len(df_cobb.index)}')
print(f'% no history cobb = {len(df4_cobb.index)/len(df1_cobb.index)}')

total cobb voters casting a ballot on 2020-11-3 = 394977
total cobb voters with no history prior to 2020-11-3 = 62007
Of the no history voters, the number of voters with opportunity for vote in 2018 general election = 31726
turnout % cobb = 0.8317669232308129
% no history cobb = 0.0803236644159027


In [78]:
from voter_history.models import VoterHistory


In [96]:
k = VoterHistory.objects.get_history('060', '20201103')
df5 = k[k.absentee]
df5.shape

(464734, 7)

In [98]:
df6 = df5[['voter_id']].merge(df4, on='voter_id', how='inner')
print(f'no history voters voting absentee or in advance = {len(df6.index)}')
print(f'% no history voters voting absentee or in advance = {len(df6.index)/len(df4.index)}')
print(f'% no history absentee fulton voters = {len(df6.index)/len(df1.index)}')

results = db.fetchall(f"""
    select av.voter_id, av.address_id
    from address_voter as av
    join voter_precinct as vp on av.voter_id = vp.voter_id
    join precinct_details pd on vp.precinct_id = pd.id
    where pd.county_code = '060'
""")
df7 = pd.DataFrame.from_records(results, columns=['voter_id', 'address_id'])
df8 = df7.merge(df6, on='voter_id', how='inner')
df9 = df8.groupby(['address_id'])['address_id'].count().sort_values(ascending=False)
print(f'% only 1 or 2 voter households = {len(df9[df9 <= 2].index)/len(df9.index)}')


no history voters voting absentee or in advance = 34130
% no history voters voting absentee or in advance = 0.7613375270472239
% no history absentee fulton voters = 0.0651944276242524
% only 1 or 2 voter households = 0.9951033002414811


In [99]:
k_cobb = VoterHistory.objects.get_history('033', '20201103')
df5_cobb = k_cobb[k_cobb.absentee]

In [101]:
df6_cobb = df5_cobb[['voter_id']].merge(df4_cobb, on='voter_id', how='inner')
print(f'no history voters voting absentee or in advance = {len(df6_cobb.index)}')
print(f'% no history voters voting absentee or in advance = {len(df6_cobb.index)/len(df4_cobb.index)}')
print(f'% no history absentee cobb voters = {len(df6_cobb.index)/len(df1_cobb.index)}')
results = db.fetchall(f"""
    select av.voter_id, av.address_id
    from address_voter as av
    join voter_precinct as vp on av.voter_id = vp.voter_id
    join precinct_details pd on vp.precinct_id = pd.id
    where pd.county_code = '060'
""")
df7_cobb = pd.DataFrame.from_records(results, columns=['voter_id', 'address_id'])
df8_cobb = df7_cobb.merge(df6_cobb, on='voter_id', how='inner')
df9_cobb = df8_cobb.groupby(['address_id'])['address_id'].count().sort_values(ascending=False)
print(f'% only 1 or 2 voter households = {len(df9_cobb[df9_cobb <= 2].index)/len(df9_cobb.index)}')

no history voters voting absentee or in advance = 22133
% no history voters voting absentee or in advance = 0.6976297043434407
% no history absentee cobb voters = 0.05603617425824795
% only 1 or 2 voter households = 1.0


In [65]:
df11.groupby(['race_id'])['race_id'].count()

race_id
AI      113
AP     2147
BH    10633
HP     1028
OT      971
U      2951
WH    12025
Name: race_id, dtype: int64